In [2]:
!pip install dgl-cu110 -f https://data.dgl.ai/wheels/repo.html

Looking in links: https://data.dgl.ai/wheels/repo.html
     |████████████████████████████████| 142.5 MB 50 kB/s 


In [3]:
import dgl
import dgl.data
from dgl.nn import GraphConv
import torch
import torch.nn as nn

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


Using backend: pytorch


In [8]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()

g = dataset[0]

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [10]:
class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.relu = nn.ReLU()
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feats):
        h = self.conv1(g, in_feats)
        h = self.relu(h)
        h = self.conv2(g, h)
        return h

In [11]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    cross_entropy = torch.nn.CrossEntropyLoss()
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']

    for e in range(100):
        logits = model(g, features)

        pred = logits.argmax(1)

        loss  = cross_entropy(logits[train_mask], labels[train_mask])

        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))

In [12]:
model = GCN(g.ndata['feat'].shape[1],
           16, 
           dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.946, val acc: 0.136 (best 0.136), test acc: 0.138 (best 0.138)
In epoch 5, loss: 1.888, val acc: 0.366 (best 0.416), test acc: 0.374 (best 0.420)
In epoch 10, loss: 1.805, val acc: 0.404 (best 0.416), test acc: 0.417 (best 0.420)
In epoch 15, loss: 1.701, val acc: 0.512 (best 0.512), test acc: 0.511 (best 0.511)
In epoch 20, loss: 1.577, val acc: 0.558 (best 0.558), test acc: 0.552 (best 0.552)
In epoch 25, loss: 1.436, val acc: 0.600 (best 0.600), test acc: 0.604 (best 0.604)
In epoch 30, loss: 1.281, val acc: 0.666 (best 0.666), test acc: 0.664 (best 0.664)
In epoch 35, loss: 1.117, val acc: 0.690 (best 0.690), test acc: 0.690 (best 0.690)
In epoch 40, loss: 0.952, val acc: 0.714 (best 0.714), test acc: 0.729 (best 0.729)
In epoch 45, loss: 0.794, val acc: 0.738 (best 0.738), test acc: 0.742 (best 0.742)
In epoch 50, loss: 0.649, val acc: 0.746 (best 0.746), test acc: 0.750 (best 0.746)
In epoch 55, loss: 0.524, val acc: 0.760 (best 0.760), test acc: 0.756 (best 0